自定义损失函数对mnist数据集的多目标线性回归实现分类任务

In [2]:
import torch
import torch.nn as nn
import numpy as np
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [3]:
class_num=10
epoch=10

In [3]:
tra=datasets.MNIST(root="data",train=True, download=True,transform=transforms.ToTensor())
tes=datasets.MNIST(root="data",train=False, download=True,transform=transforms.ToTensor())
tra_dloader = DataLoader(tra, batch_size=64, shuffle=True)
tes_dloader = DataLoader(tes, batch_size=64, shuffle=True)
print(type(tra))

<class 'torchvision.datasets.mnist.MNIST'>


D:\python\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
class mynn1(nn.Module):
    def __init__(self):
        super(mynn1, self).__init__()
        self.flatten = nn.Flatten(1,-1)
        self.fun = nn.Linear(28*28,10)
    def forward(self, x):
        x = self.flatten(x)
        logits = self.fun(x)
        return logits

In [6]:
def loss_fun(target,y):
    relu_fun=nn.ReLU()
    los=0
    tar_value=y[:,target]
    ran_long=y.size()[0]
    for i in range(0,ran_long):
        false_cla=torch.cat((y[i,0:target[i]],y[i,(target[i]+1):class_num]),0)
        false_max=torch.max(false_cla-y[i,target[i]]+1)
        los=los+relu_fun(false_max)
    return los/ran_long

In [7]:
net1=mynn1()
optimizer = torch.optim.Adam(net1.parameters())

In [8]:
for i in range(0,epoch):
    for idx,data in enumerate(tra_dloader):
        optimizer.zero_grad()
        x=data[0]
        target=data[1]
        y=net1(x)
        loss=loss_fun(target,y)
        loss.backward()
        optimizer.step()
    print(loss)

tensor(0.1706, grad_fn=<DivBackward0>)
tensor(0.4603, grad_fn=<DivBackward0>)
tensor(0.1589, grad_fn=<DivBackward0>)
tensor(0.2605, grad_fn=<DivBackward0>)
tensor(0.1297, grad_fn=<DivBackward0>)
tensor(0.3687, grad_fn=<DivBackward0>)
tensor(0.2284, grad_fn=<DivBackward0>)
tensor(0.0871, grad_fn=<DivBackward0>)
tensor(0.2007, grad_fn=<DivBackward0>)
tensor(0.2261, grad_fn=<DivBackward0>)


写一个递推计算均值函数，用于计算平均正确率

In [21]:
def mean_ditui(raw_mean,n,new_data):
    return (raw_mean*n+new_data)/(n+1)

对训练集计算预测正确率

In [23]:
total_mean=0
for idx,data in enumerate(tra_dloader):
        x=data[0]
        target=data[1]
        y=net1(x)
        pre_cla=torch.argmax(y,1)
        batch_mean=torch.mean((pre_cla==target).float())
        total_mean=mean_ditui(total_mean,idx,batch_mean)
print(total_mean)

tensor(0.9367)


测试error

In [25]:
total_mean=0
for idx,data in enumerate(tes_dloader):
        x=data[0]
        target=data[1]
        y=net1(x)
        pre_cla=torch.argmax(y,1)
        batch_mean=torch.mean((pre_cla==target).float())
        total_mean=mean_ditui(total_mean,idx,batch_mean)
print(total_mean)

tensor(0.9312)
